# Homework 3 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so don't need to load anything from disk.

In [ ]:
import random
from itertools import chain
import string
from tqdm import tqdm
import torch
import torch.nn as nn

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
  """ Converts the generated integers to one-hot vectors """
  ones = torch.sparse.torch.eye(k)
  shape = x.shape
  res = ones.index_select(0, x.view(-1).type(torch.int64)).to(device)
  return res.view(*shape, res.shape[-1])


class EchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, delay=4, seq_length=15, size=1000):
    self.delay = delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __iter__(self):
    """ Iterable dataset doesn't have to implement __getitem__.
        Instead, we only need to implement __iter__ to return
        an iterator (or generator).
    """
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
      yield seq, result

DELAY = 4
DATASET_SIZE = 10000
train_ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)
test_ds = EchoDataset(delay=DELAY, size=DATASET_SIZE//2)

In [ ]:
len(train_ds), len(test_ds)

(10000, 5000)

## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`. 

In [ ]:
class GRUMemory(torch.nn.Module):

  def __init__(self, hidden_size):
    super().__init__()
    #TODO: initialize your submodules
    # self.features = nn.Sequential(nn.Linear(27, 50), 
    # nn.ReLU(),
    # nn.Linear(50, 100),
    # nn.ReLU(),)
    self.embedding = nn.Linear(27, 5)

    self.gru = nn.GRU(5, hidden_size, 2 , bidirectional = True, batch_first = True, dropout=.9)
    
    self.classifier = nn.Sequential(nn.Linear(2*hidden_size, 75),
    nn.ReLU(),
    nn.Linear(75, 27),)


  def forward(self, x):
    # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    # TODO implement forward pass
    x = idx_to_onehot(x).to(device)
    if len(x.shape) < 3:
      x = x[None, :]
    out = self.embedding(x)    
    out = self.gru(out)[0]
    out = self.classifier(out)
    return out

  @torch.no_grad()
  def test_run(self, s):
    # This function accepts one string s containing lowercase characters a-z. 
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z.
    # ascii lower case 97-122
    ints = list(range(27))
    codes = [32]
    codes = list(chain(codes,list(range(97, 123))))
    mapping = {code:i for code, i in zip(codes, ints)}
    reversed_map = {value : key for (key, value) in mapping.items()}
    idx = torch.Tensor([mapping[int(ord(char))] for char in s])
    
    self.eval()
    self.to(device)
    res = self(idx)
    classes = res.max(dim=2)[1].squeeze().cpu()
    classes = classes.tolist()
    res = [chr(reversed_map[ind]) for ind in classes]




    


    

    # TODO
    return res

## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [ ]:
def test_model(model, sequence_length=15):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  D = DELAY
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s)
    print(s, "\n", result)
    for c1, c2 in zip(s[:-D], result[D:]):
      correct += int(c1 == c2)
      
    total += len(s) - D

  return correct / total

In [ ]:
N_EPOCH = 25
BATCH_SIZE = 64
HIDDEN_SIZE = 200
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = GRUMemory(HIDDEN_SIZE)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3, weight_decay=.00001)



 

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
train_dataloader = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, num_workers=2)
validation_dataloader = torch.utils.data.DataLoader(test_ds, batch_size=BATCH_SIZE, num_workers=2)

In [ ]:
def get_loss_and_correct(model, batch, criterion, device):
    data, target = batch
    target = target.to(device)
    pred = model(data)
    classes = pred.max(dim = 2)[1]
    pred = pred.transpose(2, 1)
    #print(pred.shape, target.shape)
    loss = criterion(pred, target)
    size1, size2 = (classes == target).shape
    correct = (classes == target).sum()
    size = size1 * size2
    return loss, correct, size


def step(loss, optimizer):
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    pass
    

In [ ]:
import time
start_time = time.time()

# TODO
train_losses = []
train_accuracies = []
validation_losses = []
validation_accuracies = []


model = model.to(device)
criterion = criterion.to(device)
model.train()
pbar = tqdm(range(N_EPOCH))

for e in pbar:
    total_train_loss = 0.0
    train_correct = []
    total_validation_loss = 0.0
    validation_correct = []
    model.train()
    for batch in train_dataloader:
        loss, correct, size = get_loss_and_correct(model, batch, criterion, device)
        step(loss, optimizer)
        total_train_loss += loss.item()
        train_correct.append((correct/size).item())
    with torch.no_grad():
        for batch in validation_dataloader:
            loss, correct, size = get_loss_and_correct(model, batch, criterion, device)
            total_validation_loss += loss.item()
            validation_correct.append((correct/size).item())

    train_len = len(train_correct)
    val_len = len(validation_correct)    
    mean_train_loss = total_train_loss / (len(train_ds))
    train_accuracy = sum(train_correct)/train_len
    mean_validation_loss = total_validation_loss / (len(test_ds))
    validation_accuracy = sum(validation_correct) /val_len
    train_losses.append(mean_train_loss)
    validation_losses.append(mean_validation_loss)

    train_accuracies.append(train_accuracy)
    validation_accuracies.append(validation_accuracy)

    pbar.set_postfix({'train_loss': mean_train_loss, 'validation_loss': mean_validation_loss, 'train_accuracy': train_accuracy, 'validation_accuracy': validation_accuracy})
    if validation_accuracy > .99:
      break

end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

 56%|█████▌    | 14/25 [01:54<01:29,  8.18s/it, train_loss=0.00131, validation_loss=0.00083, train_accuracy=0.987, validation_accuracy=0.992]


mawserksvumnbau 
 [' ', ' ', ' ', ' ', 'm', 'a', 'w', 's', 'e', 'r', 'k', 's', 'v', 'u', 'm']
oiipnawglsydgjm 
 [' ', ' ', ' ', ' ', 'o', 'i', 'i', 'p', 'n', 'a', 'w', 'g', 'l', 's', 'y']
qtnjrmdwzvxmmqdtwmz 
 [' ', ' ', ' ', ' ', 'q', 't', 'n', 'j', 'r', 'm', 'd', 'w', 'z', 'v', 'x', 'm', 'm', 'q', 'd']
tkwbdeaqhgjuvhdoumwyx 
 [' ', ' ', ' ', ' ', 't', 'k', 'w', 'b', 'd', 'e', 'a', 'q', 'h', 'g', 'j', 'u', 'v', 'h', 'd', 'o', 'u']
iwwdatuypsaafofbomwnpjtw 
 [' ', ' ', ' ', ' ', 'i', 'w', 'w', 'd', 'a', 't', 'u', 'y', 'p', 's', 'a', 'a', 'f', 'o', 'f', 'b', 'o', 'm', 'w', 'n']
khnwjrljpmzrfnenx 
 [' ', ' ', ' ', ' ', 'k', 'h', 'n', 'w', 'j', 'r', 'l', 'j', 'p', 'm', 'z', 'r', 'f']
whvjgdkycsrnxvvmu 
 [' ', ' ', ' ', ' ', 'w', 'h', 'v', 'j', 'g', 'd', 'k', 'y', 'c', 's', 'r', 'n', 'x']
pzukexmzlaiputmhozwzexsc 
 [' ', ' ', ' ', ' ', 'p', 'z', 'u', 'k', 'e', 'x', 'm', 'z', 'l', 'a', 'i', 'p', 'u', 't', 'm', 'h', 'o', 'z', 'w', 'z']
emneetupjhkpckisyqnbebpln 
 [' ', ' ', ' ', ' ', 'e', 'm

In [ ]:
for batch in validation_dataloader:
  x, y = batch 
  print(model(x).max(dim))

78.91815400123596

## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay stays constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [ ]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, max_delay=8, seq_length=20, size=1000):
    self.max_delay = max_delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __iter__(self):
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      delay = random.randint(0, self.max_delay)
      result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
      yield seq, delay, result

### Model

And the model.

In [ ]:
class VariableDelayGRUMemory(torch.nn.Module):

  def __init__(self, hidden_size, max_delay):
    super().__init__()
    #TODO
    self.max_delay = max_delay

    self.embedding = nn.Linear(28, 10)
    self.gru = nn.GRU(10, hidden_size, 1 , bidirectional = True, batch_first = True, dropout=.9, )
    
    self.classifier = nn.Sequential(nn.Linear(2*hidden_size, 75),
    nn.ReLU(),
    nn.Linear(75, 27))


  def forward(self, x, delay):
    # inputs:
    # x - tensor of shape (batch size, seq length, N + 1)
    # delays - tensor of shape (batch size)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    x = idx_to_onehot(x).to(device)
    if len(x.shape) < 3:
      x = x[None, :, :]
      delay = torch.Tensor([delay])
    #delay = idx_to_onehot(delays, k=self.max_delay+1).to(device).unsqueeze(dim=1).expand(-1, 20, -1)
    delay = delay[:, None, None].expand(-1, 20, -1).to(device)
    input = torch.cat((delay, x), dim =2)
    out = self.embedding(input)
    out = self.gru(out)[0]
    out = self.classifier(out)
    print
    # TODO
    return out

  @torch.no_grad()
  def test_run(self, s, delay):
    # This function accepts one string s containing lowercase characters a-z, 
    # and a delay - the desired output delay.
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z. 
    ints = list(range(27))
    codes = [32]
    codes = list(chain(codes,list(range(97, 123))))
    mapping = {code:i for code, i in zip(codes, ints)}
    reversed_map = {value : key for (key, value) in mapping.items()}
    idx = torch.Tensor([mapping[int(ord(char))] for char in s])
    
    model.eval()
    model.to(device)
    res = model(idx, delay)
    classes = res.max(dim=2)[1].squeeze().cpu()
    classes = classes.tolist()
    res = [chr(reversed_map[ind]) for ind in classes]




    


    # TODO
    return res

In [ ]:

for batch in train_1:
  x, y, z = batch 
  break

y = y[:, None, None].expand(-1, 20, -1).to(device)
one_hot_x = idx_to_onehot(x)
# torch.cat((one_hot, one_hot_x))
print(one_hot_x.shape, one_hot_y.shape)
new = torch.cat((one_hot_x, y), dim=2)
new.shape

torch.Size([128, 20, 27]) torch.Size([64, 20, 9])


torch.Size([128, 20, 28])

In [ ]:
def idx_to_padding(x, k=N+1):
  """ Converts the generated integers to one-hot vectors """
  ones = torch.zeros((64, 20, k))
  shape = x.shape
  res = ones.index_select(2, x.view(-1).type(torch.int64))
  return res.view(*shape, res.shape[-1])

### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [ ]:
def loss_correct(model, batch, loss_criteria, device):
  data, delay, target = batch
  target= target.to(device)
  preds = model(data, delay)
  classes = preds.max(dim=2)[1]
  preds = preds.transpose(1,2)
  loss = loss_criteria(preds, target)  
  size1, size2 = (classes == target).shape
  correct = (classes == target).sum()
  size = size1 * size2
  return loss, correct, size

def update(loss, optimizer):
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()


In [ ]:
def test_variable_delay_model(model, seq_length=20):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(seq_length)])
    d = random.randint(0, model.max_delay)
    result = model.test_run(s, d)

    print(s, "\n", result, d)
    if d > 0:
      z = zip(s[:-d], result[d:])
    else:
      z = zip(s, result)
    for c1, c2 in z:
      correct += int(c1 == c2)
    total += len(s) - d

  return correct / total

In [ ]:
import time
start_time = time.time()
BATCH_LEN = 128
MAX_DELAY = 8
SEQ_LENGTH = 20
HID_DIM = 200
EPOCHS = 80
train_1 = torch.utils.data.DataLoader(VariableDelayEchoDataset(max_delay=MAX_DELAY, seq_length=SEQ_LENGTH, size=10000), batch_size=BATCH_LEN, num_workers=2)
valid_1 = torch.utils.data.DataLoader(VariableDelayEchoDataset(max_delay=MAX_DELAY, seq_length=SEQ_LENGTH, size=1000),batch_size=BATCH_LEN, num_workers=2)
# TODO: implement model training here.

model = VariableDelayGRUMemory(HID_DIM, max_delay=MAX_DELAY)
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr = 1e-3)

train_losses = []
train_accuracies = []
validation_losses = []
validation_accuracies = []

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)
criterion = criterion.to(device)
model.train()
pbar = tqdm(range(EPOCHS))

for e in pbar:
    total_train_loss = 0.0
    train_correct = []
    total_validation_loss = 0.0
    validation_correct = []
    model.train()
    for batch in train_1:
        loss, correct, size = loss_correct(model, batch, criterion, device)
        update(loss, optim)
        total_train_loss += loss.item()
        train_correct.append((correct/size).item())
    with torch.no_grad():
        for batch in valid_1:
            loss, correct, size = loss_correct(model, batch, criterion, device)
            total_validation_loss += loss.item()
            validation_correct.append((correct/size).item())

    train_len = len(train_correct)
    val_len = len(validation_correct)    
    mean_train_loss = total_train_loss / (len(train_ds))
    train_accuracy = sum(train_correct)/train_len
    mean_validation_loss = total_validation_loss / (len(test_ds))
    validation_accuracy = sum(validation_correct) /val_len
    train_losses.append(mean_train_loss)
    validation_losses.append(mean_validation_loss)

    train_accuracies.append(train_accuracy)
    validation_accuracies.append(validation_accuracy)

    pbar.set_postfix({'train_loss': mean_train_loss, 'validation_loss': mean_validation_loss, 'train_accuracy': train_accuracy, 'validation_accuracy': validation_accuracy})
    if validation_accuracy > .99:
      break

end_time = time.time()
assert end_time - start_time < 600, 'executing took longer than 10 mins'
assert test_variable_delay_model(model) > 0.99, 'accuracy is too low'
print('tests passed')

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.9 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
 65%|██████▌   | 52/80 [03:26<01:51,  3.97s/it, train_loss=0.000909, validation_loss=7.46e-5, train_accuracy=0.982, validation_accuracy=0.995]


mwfncizniglczylltiyt 
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'm', 'w', 'f', 'n', 'd', 'i', 'z', 'n', 'i', 'g', 'l', 'c'] 8
divnyuzkzuheebvbtsvi 
 [' ', ' ', ' ', 'd', 'i', 'v', 'n', 'y', 'u', 'z', 'k', 'z', 'u', 'h', 'e', 'e', 'b', 'v', 'b', 't'] 3
smvttapcdivcatqiqqmz 
 [' ', 's', 'm', 'v', 't', 't', 'a', 'p', 'c', 'd', 'i', 'v', 'c', 'a', 't', 'q', 'i', 'q', 'q', 'm'] 1
eltkgvwlrnnepdhmuqhv 
 [' ', ' ', ' ', ' ', ' ', ' ', 'e', 'l', 't', 'k', 'g', 'v', 'w', 'l', 'r', 'n', 'n', 'e', 'p', 'd'] 6
ntzxdowfcxhzjqsnsxoy 
 [' ', ' ', ' ', ' ', ' ', ' ', 'n', 't', 'z', 'x', 'd', 'i', 'w', 'f', 'c', 'x', 'h', 'z', 'j', 'q'] 6
bposnwbrajrbbulwpnfj 
 [' ', ' ', ' ', ' ', 'b', 'p', 'o', 's', 'n', 'w', 'b', 'r', 'a', 'j', 'r', 'b', 'b', 'u', 'l', 'w'] 4
xiykxdtrlcoglrhtinra 
 ['x', 'i', 'y', 'k', 'x', 'd', 't', 'r', 'l', 'c', 'o', 'g', 'l', 'r', 'h', 't', 'i', 'n', 'r', 'a'] 0
jllvicefdtrsgelkgsdc 
 [' ', ' ', ' ', 'j', 'l', 'l', 'v', 'i', 'c', 'e', 'f', 'd', 't', 'r', 's', 'g', 'e', 'l', 'k',